In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-144341
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-144341


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# Create compute cluster
# Compute cluster name
compute_cluster_name = "compute-cluster"

# Use vm_size = "Standard_D2_V2" in your provisioning configuration
vm_size = "Standard_D2_V2"

# max_nodes should be no greater than 4
max_nodes = 4

# Verify that cluster does not exist already
try:
    compute_cluster =  ComputeTarget(workspace=ws, name=compute_cluster_name)
    print(f"Compute cluster '{compute_cluster_name}' already exists, use it instead of trying to create a new cluster with the same name.")
except:
    compute_cluster_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=max_nodes)
    compute_cluster = ComputeTarget.create(workspace=ws, name=compute_cluster_name, provisioning_configuration=compute_cluster_config)

compute_cluster.wait_for_completion(show_output=True)

Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os, shutil

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.001, 100),
    "--max_iter": choice(10, 50, 100, 150, 200)
    })

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

# Create training directory
train_dir = "./training"
train_script = "train.py"

if not os.path.exists(train_dir):
    os.mkdir(train_dir)

# Copy train script
shutil.copy(train_script, train_dir)

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory=train_dir,
    entry_script=train_script,
    compute_target=compute_cluster
    )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=16,
    max_concurrent_runs=4,
    policy=policy,
    )

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
hyperdrive_run

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e2b5fc0b-07d4-44b7-8c61-89173e61efc1
Web View: https://ml.azure.com/runs/HD_e2b5fc0b-07d4-44b7-8c61-89173e61efc1?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-144341/workspaces/quick-starts-ws-144341&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-05-10T06:26:27.253356][API][INFO]Experiment created<END>\n""<START>[2021-05-10T06:26:27.756605][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-05-10T06:26:27.986548][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_e2b5fc0b-07d4-44b7-8c61-89173e61efc1
Web View: https://ml.azure.com/runs/HD_e2b5fc0b-07d4-44b7-8c61-89173e61efc1?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-144341/workspaces/quick-starts-ws-144341&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_e2b5fc0b-07d4-44b7-8c61-89173e61efc1,hyperdrive,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [5]:
import joblib

# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(f"Best run arguments: {best_run.get_details()['runDefinition']['arguments']}")
print(f"Best run metrics: {best_run.get_metrics()}")
print(f"Best run file names: {best_run.get_file_names()}")
joblib.dump(value=best_run.id, filename="./outputs/hyperdrive_model.joblib")

Best run arguments: ['--C', '12.436625838582243', '--max_iter', '150']
Best run metrics: {'Regularization Strength:': 12.436625838582243, 'Max iterations:': 150, 'Accuracy': 0.9096611026808296}
Best run file names: ['azureml-logs/55_azureml-execution-tvmps_fbb0a0703d720551faa08b0e1536c90a47721aaaf40891d928594b74ea788acb_d.txt', 'azureml-logs/65_job_prep-tvmps_fbb0a0703d720551faa08b0e1536c90a47721aaaf40891d928594b74ea788acb_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_fbb0a0703d720551faa08b0e1536c90a47721aaaf40891d928594b74ea788acb_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/106_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']


['./outputs/hyperdrive_model.joblib']

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=path)

In [7]:
import pandas as pd
from train import clean_data

# Use the clean_data function to clean your data
x, y = clean_data(ds)

train_df = pd.concat([x, y], axis=1)
train_df_path = os.path.join(train_dir, 'bankmarketing_train.csv')
train_df.to_csv(train_df_path)
data_store = ws.get_default_datastore()
data_store.upload(src_dir=train_dir, target_path=train_dir)
train_ds = TabularDatasetFactory.from_delimited_files(data_store.path(train_df_path))

Uploading an estimated of 2 files
Target already exists. Skipping upload for training/bankmarketing_train.csv
Target already exists. Skipping upload for training/train.py
Uploaded 0 files


In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    name="AutoML pipeline",
    compute_target=compute_cluster,
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_ds,
    label_column_name="y",
    n_cross_validations=5
    )

In [9]:
# Submit your AutoML run
automl_run = exp.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)
automl_run

Submitting remote run.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_4e96c1ad-3437-4b86-8f75-00b447aea560,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_4e96c1ad-3437-4b86-8f75-00b447aea560,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          0:00:54       0.9152    0.9152
         1   MaxAbsScaler XGBoostClassifier                 0:01:01       0.9156    0.9156
         2   MaxAbsScaler RandomForest                      0:00:52       0.8933    0.9156
         3   MaxAbsScaler RandomForest                      0:00:51       0.8880    0.9156
         4   MaxAbsScaler RandomForest                      0:00:58       0.8018    0.9156
         5   MaxAbsScaler RandomForest                      0:00:49       0.7614    0.9156
         6   SparseNormalizer XGBoostClassifier             0:01:40       0.9124    0.9156
         7   MaxAbsS

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_4e96c1ad-3437-4b86-8f75-00b447aea560,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [10]:
# Retrieve and save your best automl model.
best_run, best_model = automl_run.get_output()
print(best_run)
print(best_model)
print(f"Best run arguments: {best_run.get_details()['runDefinition']['arguments']}")
print(f"Best run metrics: {best_run.get_metrics()}")
print(f"Best run file names: {best_run.get_file_names()}")
joblib.dump(best_model, 'outputs/automl_model.joblib')

Run(Experiment: udacity-project,
Id: AutoML_4e96c1ad-3437-4b86-8f75-00b447aea560_22,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               nthread=None,
                                                                                               objective='reg:logistic',
                                 

['outputs/automl_model.joblib']

In [11]:
# Clean up
compute_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

